In [1]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import sklearn
from sklearn import preprocessing
from collections import defaultdict

### Importing Data and Basic Analysis

In [2]:
import csv
df_raw = pd.read_csv('data/M_query_20200512.csv', sep=',',skipinitialspace=True,
                    engine='python', error_bad_lines=False,quoting=csv.QUOTE_ALL, warn_bad_lines=False)

In [3]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 547854 entries, 0 to 547853
Data columns (total 23 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   TrialId                  547854 non-null  int64  
 1   TrialGroupId             547854 non-null  int64  
 2   TrialTargetId            547854 non-null  int64  
 3   TrialPhase               547854 non-null  object 
 4   TrialValue               547854 non-null  float64
 5   Trial                    547854 non-null  int64  
 6   TrialCreatedDate         547854 non-null  object 
 7   TrialDataDate            547854 non-null  object 
 8   TrialAuthorId            547854 non-null  int64  
 9   GoalName                 547854 non-null  object 
 10  ClientId                 547854 non-null  int64  
 11  GoalType                 547854 non-null  object 
 12  CurrentGoalStatus        547854 non-null  object 
 13  GoalDomain               547854 non-null  object 
 14  Goal

In [5]:
df_raw.replace("?",np.nan,inplace=True)

In [68]:
#helper function
def success_rate(sumval, total):
    return (sumval/total)*100

In [69]:
def normalize_values(df):
    normalized_df=(df-df.min())/(df.max()-df.min())
    return normalized_df

In [81]:
df_sum = df_raw.groupby(['ClientId','TrialDataDate','GoalDomain','TrialAuthorId','CurrentGoalStatus'],as_index=False)['TrialValue'].agg('sum')
df_total = df_raw.groupby(['ClientId','TrialDataDate','GoalDomain','TrialAuthorId','CurrentGoalStatus']).size().reset_index(name='TrialsPerDatePerAuthor')


In [82]:
df_total.head()

,ClientId,TrialDataDate,GoalDomain,TrialAuthorId,CurrentGoalStatus,TrialsPerDatePerAuthor
0,88550,2018-02-26 15:54:00,Language,470123,Met,3
1,88550,2018-03-01 15:53:00,Language,470123,Met,5
2,88550,2018-03-05 15:51:00,Language,470123,Met,3
3,88550,2018-03-09 06:45:00,Language,344083,Met,6
4,88550,2018-03-16 07:42:00,Language,344083,Met,5


In [83]:
#df_sum['TrialValue_norm'] = normalize_values(df_sum['TrialValue'])
#df_total['TrialsPerDatePerAuthor_norm'] = normalize_values(df_total['TrialsPerDatePerAuthor'])

In [84]:
df_sum['TrialValue'] = df_sum['TrialValue']
df_total['TrialsPerDatePerAuthor'] = df_total['TrialsPerDatePerAuthor']

In [85]:
df_sum.head(5)

,ClientId,TrialDataDate,GoalDomain,TrialAuthorId,CurrentGoalStatus,TrialValue
0,88550,2018-02-26 15:54:00,Language,470123,Met,0.0
1,88550,2018-03-01 15:53:00,Language,470123,Met,4.0
2,88550,2018-03-05 15:51:00,Language,470123,Met,0.0
3,88550,2018-03-09 06:45:00,Language,344083,Met,2.0
4,88550,2018-03-16 07:42:00,Language,344083,Met,2.0


In [86]:
df_total.head(5)

,ClientId,TrialDataDate,GoalDomain,TrialAuthorId,CurrentGoalStatus,TrialsPerDatePerAuthor
0,88550,2018-02-26 15:54:00,Language,470123,Met,3
1,88550,2018-03-01 15:53:00,Language,470123,Met,5
2,88550,2018-03-05 15:51:00,Language,470123,Met,3
3,88550,2018-03-09 06:45:00,Language,344083,Met,6
4,88550,2018-03-16 07:42:00,Language,344083,Met,5


In [87]:
df_sum['SuccessPercentage'] = success_rate((df_sum['TrialValue']),(df_total['TrialsPerDatePerAuthor']))
                                           

In [75]:
#df_sum['SuccessPercentage_norm'] = normalize_values(success_rate((df_sum['TrialValue_norm']),(df_total['TrialsPerDatePerAuthor_norm'])))
#df_sum['SuccessPercentage'] = normalize_values(df_sum['SuccessPercentage_raw'])                                          

In [89]:
#df_sum.head(20)

In [96]:
df_tot_cases = df_total.groupby(['ClientId']).size().reset_index(name='TotalClientCases')
df_totalclient_suc = df_sum.groupby(['ClientId'],as_index=False)['SuccessPercentage'].mean()

#### Identify clients with low success rate and see if it is possible to refer to domain experts of the problem area

In [97]:
df_totalclient_suc.tail()

,ClientId,SuccessPercentage
1208,1264327,96.527778
1209,1272784,19.333333
1210,1275765,95.000000
1211,1278577,27.525253
1212,1282868,66.666667


In [100]:
df_totalclient_suc.sort_values(by=['SuccessPercentage'],ascending=False).tail(5)

,ClientId,SuccessPercentage
1154,1124604,0.0
1079,1044572,0.0
673,687874,0.0
729,718414,0.0
856,819281,0.0


#### Let's check out the clientId 718414 with zero success rate. Let's grab the TraiAuthorId to see the expertise of 

In [108]:
df_raw[df_raw.ClientId==819281].head(5)

,TrialId,TrialGroupId,TrialTargetId,TrialPhase,TrialValue,Trial,TrialCreatedDate,TrialDataDate,TrialAuthorId,GoalName,...,GoalDomain,GoalAssessment,GoalCreatedDate,GoalInitiatedDate,GoalMetDate,GoalInProgressDate,GoalHoldDate,GoalDiscontinuedDate,GoalDataType,GoalPercentCorrectTrend
152954,2325064478,533873395,81407697,baseline,0.0,1,2019-12-11 17:41:30.597000000,2019-12-11 12:41:00,1110314,Goes to desk to work on ind work,...,Learning Readiness,Verbal Behavior Milestone Assessment and Place...,2019-12-11 16:13:36.863000000,2019-12-11,NaN,2019-12-11,NaN,NaN,datapercent,0.0
152955,2325064479,533873395,81407697,baseline,0.0,2,2019-12-11 17:41:30.597000000,2019-12-11 12:41:00,1110314,Goes to desk to work on ind work,...,Learning Readiness,Verbal Behavior Milestone Assessment and Place...,2019-12-11 16:13:36.863000000,2019-12-11,NaN,2019-12-11,NaN,NaN,datapercent,0.0
152956,2325064480,533873395,81407697,baseline,0.0,3,2019-12-11 17:41:30.597000000,2019-12-11 12:41:00,1110314,Goes to desk to work on ind work,...,Learning Readiness,Verbal Behavior Milestone Assessment and Place...,2019-12-11 16:13:36.863000000,2019-12-11,NaN,2019-12-11,NaN,NaN,datapercent,0.0
152957,2325654310,534011102,81407697,Intervention,0.0,1,2019-12-11 18:43:13.970000000,2019-12-11 13:43:00,1110314,Goes to desk to work on ind work,...,Learning Readiness,Verbal Behavior Milestone Assessment and Place...,2019-12-11 16:13:36.863000000,2019-12-11,NaN,2019-12-11,NaN,NaN,datapercent,0.0
152958,2325654311,534011102,81407697,Intervention,0.0,2,2019-12-11 18:43:13.970000000,2019-12-11 13:43:00,1110314,Goes to desk to work on ind work,...,Learning Readiness,Verbal Behavior Milestone Assessment and Place...,2019-12-11 16:13:36.863000000,2019-12-11,NaN,2019-12-11,NaN,NaN,datapercent,0.0


### Ranking expertise of authors in a specific Goal Domain

In [27]:
df_tot_cases_author = df_total.groupby(['TrialAuthorId','GoalDomain']).size().reset_index(name='TotalClientCases')
df_totalauthor_suc = df_sum.groupby(['TrialAuthorId','GoalDomain'],as_index=False)['SuccessPercentage'].mean()

In [28]:
df_totalauthor_suc.head(5)

,TrialAuthorId,GoalDomain,SuccessPercentage
0,81006,Academic,10.000000
1,81852,Communication,75.555556
2,81852,Imitation,26.666667
3,81852,Learning Readiness,61.111111
4,95238,Language,89.583333


### Ranking experts based on goal domain

In [33]:
df_auth_rank_pre = df_totalauthor_suc.sort_values(by=['SuccessPercentage'],ascending = False)

In [34]:
df_auth_rank = df_auth_rank_pre[['GoalDomain', 'TrialAuthorId', 'SuccessPercentage']].sort_values(by=['GoalDomain'])

In [35]:
df_domain_rank = df_auth_rank.sort_values(by=['SuccessPercentage'],ascending = False)

In [38]:
df_domain_rank.head()

,GoalDomain,TrialAuthorId,SuccessPercentage
1912,Imitation,561685,100.0
1424,Communication,508433,100.0
4256,Communication,886942,100.0
1280,Communication,485433,100.0
1640,Communication,528867,100.0


In [106]:
df_domain_rank[df_domain_rank.TrialAuthorId==1110314]

,GoalDomain,TrialAuthorId,SuccessPercentage
5536,Learning Readiness,1110314,0.0


#### As we can see above the "TraiAuthorId" of the specific client with low success percentage is zero

##### Now, let's check which providers / AuthorId are having high success percentage in this Goal Domain

In [109]:
df_domain_rank[df_domain_rank.GoalDomain=="Learning Readiness"].head(5)

,GoalDomain,TrialAuthorId,SuccessPercentage
1333,Learning Readiness,492655,100.0
1448,Learning Readiness,508473,100.0
1446,Learning Readiness,508468,100.0
1503,Learning Readiness,513709,100.0
3971,Learning Readiness,845557,100.0


### This work is based on preliminary exploratory analysis and demonstrates a suitable feature for a Machine Learning system to be able to recommend right service providers based on the expertise.